In [ ]:
from sshtunnel import SSHTunnelForwarder 
from dotenv import load_dotenv
import os
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from sshtunnel import SSHTunnelForwarder 
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from dotenv import load_dotenv
import os
import pandas as pd
import plotly.express as px

In [2]:
load_dotenv()

True

In [4]:
def run_query(query_str):
    with SSHTunnelForwarder(
        (os.getenv("BASTION_SERVER_IP"), 22), #Remote server IP and SSH port
        ssh_username = os.getenv("SSH_USER_NAME"),
        ssh_pkey=os.getenv("SSH_PRIVATE_KEY_PATH"),
        remote_bind_address=(os.getenv("RDS_ENDPOINT"), 3306)) as server: 
            
        server.start() #start ssh sever
        # print('Server connected via SSH')
        
        #connect to MariaDB
        local_port = str(server.local_bind_port)
        database_name = 'financial'
        engine = create_engine(
            'mariadb+mariadbconnector://' 
            + os.getenv('DB_USERNAME') 
            + ':' 
            + os.getenv('DB_PASSWORD') 
            + '@127.0.0.1:'
            + local_port 
            + '/' 
            + database_name
        )

        Session = sessionmaker(bind=engine)
        session = Session()
        
        # print('Database session created')
        
        #test data retrieval
        test = session.execute(text(query_str))
        df = pd.DataFrame.from_records(test)
            
        session.close()

        return df

In [20]:
cards_df = run_query('SELECT * FROM card LIMIT 5')
cards_df.columns = run_query('SHOW COLUMNS FROM card')[0].tolist()

loan_df = run_query('SELECT * FROM loan LIMIT 5')
loan_df.columns = run_query('SHOW COLUMNS FROM loan')[0].tolist()

In [21]:
agent = create_pandas_dataframe_agent(
  ChatGoogleGenerativeAI(model='gemini-2.0-flash-001'), 
  [cards_df, loan_df], 
  verbose=True,
  allow_dangerous_code=True
)

In [25]:
agent.invoke("plot count by card type")



> Entering new AgentExecutor chain...
Thought: I want to create a count plot of the 'type' column from the first dataframe.
Action: [python_repl_ast]
Action Input:
```python
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data1 = {'card_id': [1, 2, 3, 4, 5], 'disp_id': [9, 19, 41, 42, 51], 'type': ['gold', 'classic', 'gold', 'classic', 'junior'], 'issued': ['1998-10-16', '1998-03-13', '1995-09-03', '1998-11-26', '1995-04-24']}
df = pd.DataFrame(data1)

sns.countplot(x='type', data=df)
plt.show()
```[python_repl_ast] is not a valid tool, try one of [python_repl_ast].I made a mistake and used the wrong tool. I need to execute the plotting command in the python_repl_ast tool.
Action: [python_repl_ast]
Action Input:
```python
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data1 = {'card_id': [1, 2, 3, 4, 5], 'disp_id': [9, 19, 41, 42, 51], 'type': ['gold', 'classic', 'gold', 'classic', 'junior'], 'issued': ['1998-10-16', '1998-03-13'

{'input': 'plot count by card type',
 'output': "A countplot of the 'type' column from the first dataframe (df1) would show the counts of each card type (gold, classic, junior)."}